### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [1]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [2]:
url = "http://www.allocine.fr/"
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [3]:
for p in soup.find_all("a"):
    print(p.text)

Meilleurs films d'animation Disney
Meilleurs films Pixar

            Cinéma
        

            Séries
        

            Streaming
        

            Trailers
        

            VOD
        

            Les indés
        
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces
Agenda des sorties
Kids
Meilleurs films
Meilleurs documentaires
Avant-premières
Box Office
Films pour enfants à l'affiche
DVD
Meilleurs films 2019
Tous les films pour enfants
Tous les films
Sorties de la semaine
Monsieur Aznavour
Transformers : le commencement
4 zéros
Challenger
Angelo dans la forêt mystérieuse
The Killer
Trois kilomètres jusqu'à la fin du monde
Fario
Carla et Moi
Chroniques chinoises
Le Robot Sauvage
Joker: Folie à Deux
L'Amour ouf
Lee Miller
Tous les films encore à l'affiche
 Juré n°2 
 
 Venom: The Last Dance 
 
 Anora 
 
 Gladiator II 
 
 Sur un fil 
 
Tous les films à venir
Dernières news cinéma
"Mais quel acteur, quel p**** d'acteur, quelle force !" : Alban I

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [4]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=313486.html" title="Monsieur Aznavour">Monsieur Aznavour</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=241064.html" title="Transformers : le commencement">Transformers : le commencement</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=304852.html" title="4 zéros">4 zéros</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=317949.html" title="Challenger">Challenger</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=299773.html" title="Angelo dans la forêt mystérieuse">Angelo dans la forêt mystérieuse</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=131724.html" title="The Killer">The Killer</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000001770.html" title="Trois kilomètres jusqu'à la fin du monde">Trois kilomètres jusqu'à la fin du monde</a>
<a class="meta-title meta

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [7]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

/film/fichefilm_gen_cfilm=313486.html
/film/fichefilm_gen_cfilm=241064.html
/film/fichefilm_gen_cfilm=304852.html
/film/fichefilm_gen_cfilm=317949.html
/film/fichefilm_gen_cfilm=299773.html
/film/fichefilm_gen_cfilm=131724.html
/film/fichefilm_gen_cfilm=1000001770.html
/film/fichefilm_gen_cfilm=302436.html
/film/fichefilm_gen_cfilm=324837.html
/film/fichefilm_gen_cfilm=1000000659.html
/film/fichefilm_gen_cfilm=268384.html
/film/fichefilm_gen_cfilm=278006.html
/film/fichefilm_gen_cfilm=221617.html
/film/fichefilm_gen_cfilm=259096.html
/series/ficheserie_gen_cserie=34726.html
/series/ficheserie_gen_cserie=34229.html
/series/ficheserie_gen_cserie=33768.html
/series/ficheserie_gen_cserie=34107.html
/series/ficheserie_gen_cserie=25210.html
/series/ficheserie_gen_cserie=34642.html
/series/ficheserie_gen_cserie=34895.html


Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

In [9]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("title"))


#for elem in soup.find_all("title"):
#    print(elem.text)

Monsieur Aznavour
Transformers : le commencement
4 zéros
Challenger
Angelo dans la forêt mystérieuse
The Killer
Trois kilomètres jusqu'à la fin du monde
Fario
Carla et Moi
Chroniques chinoises
Le Robot Sauvage
Joker: Folie à Deux
L'Amour ouf
Lee Miller
Culte
Les enfants sont rois
Le Daron
Cat's Eyes
Dune: Prophecy
Murder Club
La Cage


#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [10]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links

['/film/fichefilm_gen_cfilm=313486.html',
 '/film/fichefilm_gen_cfilm=241064.html',
 '/film/fichefilm_gen_cfilm=304852.html',
 '/film/fichefilm_gen_cfilm=317949.html',
 '/film/fichefilm_gen_cfilm=299773.html',
 '/film/fichefilm_gen_cfilm=131724.html',
 '/film/fichefilm_gen_cfilm=1000001770.html',
 '/film/fichefilm_gen_cfilm=302436.html',
 '/film/fichefilm_gen_cfilm=324837.html',
 '/film/fichefilm_gen_cfilm=1000000659.html',
 '/film/fichefilm_gen_cfilm=268384.html',
 '/film/fichefilm_gen_cfilm=278006.html',
 '/film/fichefilm_gen_cfilm=221617.html',
 '/film/fichefilm_gen_cfilm=259096.html',
 '/series/ficheserie_gen_cserie=34726.html',
 '/series/ficheserie_gen_cserie=34229.html',
 '/series/ficheserie_gen_cserie=33768.html',
 '/series/ficheserie_gen_cserie=34107.html',
 '/series/ficheserie_gen_cserie=25210.html',
 '/series/ficheserie_gen_cserie=34642.html',
 '/series/ficheserie_gen_cserie=34895.html']

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [11]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

['http://www.allocine.fr/film/fichefilm_gen_cfilm=313486.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=241064.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=304852.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=317949.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=299773.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=131724.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000001770.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=302436.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=324837.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000000659.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=268384.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=278006.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=221617.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=259096.html']

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [12]:
url = movie_links[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=313486.html 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1,max-image-preview:standard" name="robots"/>
<title>Monsieur Aznavour - Film 2024 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Monsieur Aznavour est un film réalisé par Mehdi Idir et Grand Corps Malade avec Tahar Rahim, Bastien Bouillon. Synopsis : Fils de réfugiés, petit, pauvre, à la voix voilée, on disait de lui qu’

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<section class="section ovw ovw-synopsis" id="synopsis-details">

      <div class="content-txt">
 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
      </div>
    
      </section>
```

In [13]:
for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-lg"}):
    # Just like that
    print(elem.text)

# Get the synopsis section
for elem in soup.find_all("section", attrs={"id": "synopsis-details"}):
    # Get the text of the synopsis
    for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
        # Just like that
        print(elem2.text)


Fils de réfugiés, petit, pauvre, à la voix voilée, on disait de lui qu’il n’avait rien pour réussir. À force de travail, de persévérance et d’une volonté hors norme, Charles Aznavour est devenu un monument de la chanson, et un symbole de la culture française. Avec près de 1200 titres interprétés dans le monde entier et dans toutes les langues, il a inspiré des générations entières. Découvrez le parcours exceptionnel et intemporel de MONSIEUR AZNAVOUR.



1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [14]:
import time
import random
from random import randint

title = []
synopsis = []

# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis

# Check the length of the lists before creating the dataframe
len(title), len(synopsis), len(movie_links)

(0, 0, 14)

In [15]:
import pandas as pd

df = pd.DataFrame({"Title": title})
df["Synopsis"] = synopsis
df["Links"] = movie_links

In [16]:
df.to_csv("./assets/allo_cine.csv", index=False)